# Assignment 7 - Dialog Systems and Question Answering

- Student 1 - Luca Modica
- Student 2 - Hugo Alves Henriques E Silva

---

In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from icecream import ic
import random


In [41]:


sns.set_style()
%matplotlib inline
np.random.seed(0)

Example from chatGPT

In [42]:
# import requests

# # API endpoints
# weather_api_endpoint = "https://api.weather.com/forecast"
# restaurant_api_endpoint = "https://api.yelp.com/v3/businesses/search"
# transportation_api_endpoint = "https://api.transportation.com/schedule"

# # API keys
# weather_api_key = "your_weather_api_key"
# restaurant_api_key = "your_restaurant_api_key"
# transportation_api_key = "your_transportation_api_key"

# def get_weather_forecast(city):
#     params = {
#         "apikey": weather_api_key,
#         "city": city,
#         "days": 1
#     }
#     response = requests.get(weather_api_endpoint, params=params)
#     data = response.json()
#     forecast = data["forecast"]["daily"]["summary"]
#     return forecast

# def find_restaurant(cuisine):
#     headers = {
#         "Authorization": f"Bearer {restaurant_api_key}"
#     }
#     params = {
#         "term": cuisine,
#         "location": "your_location"
#     }
#     response = requests.get(restaurant_api_endpoint, headers=headers, params=params)
#     data = response.json()
#     businesses = data["businesses"]
#     if businesses:
#         return random.choice(businesses)["name"]
#     else:
#         return None

# def get_next_tram_bus(mode):
#     headers = {
#         "Authorization": f"Bearer {transportation_api_key}"
#     }
#     params = {
#         "mode": mode,
#         "location": "your_location"
#     }
#     response = requests.get(transportation_api_endpoint, headers=headers, params=params)
#     data = response.json()
#     schedule = data["schedule"]
#     if schedule:
#         return random.choice(schedule)["time"]
#     else:
#         return None

# # Rest of the code remains the same as the previous implementation

In [43]:
import pandas as pd

df_restaurants = pd.read_csv("data/restaurants.csv")
df_restaurants.head()

,City,Restaurant,Street,Rating
0,London,Finnish,Lily Place 763,4.7
1,London,Croatian,Cherry Way 611,3.5
2,London,South African,Moonlight Avenue 181,4.5
3,Paris,Finnish,Peach Boulevard 803,4.5
4,Paris,Belgian,Rainbow Road 205,4.2


In [44]:
df_flights = pd.read_csv("data/flights.csv")

ic(df_flights.shape)
df_flights.head()

ic| df_flights.shape: (12250, 4)


,FlightNumber,Origin,Destination,Departure
0,EU0001,London,Paris,06:00
1,EU0002,London,Paris,10:00
2,EU0003,London,Paris,14:00
3,EU0004,London,Paris,18:00
4,EU0005,London,Paris,22:00


In [45]:
# Read the parquet file
df_weather = pd.read_parquet('data/daily_weather.parquet')
df_cities = pd.read_csv('data/cities.csv')

#remove entries from df_weather where the year is not 2023
df_weather = df_weather[df_weather['date'].dt.year == 2023]

# Display the head of the dataframe
print(df_weather.head())
print(df_cities.head())


     station_id city_name       date  season  avg_temp_c  min_temp_c  \
9332      41515  Asadabad 2023-01-01  Winter         5.7        -1.0   
9333      41515  Asadabad 2023-01-02  Winter         5.0        -1.4   
9334      41515  Asadabad 2023-01-03  Winter         4.9        -1.2   
9335      41515  Asadabad 2023-01-04  Winter         5.6        -1.4   
9336      41515  Asadabad 2023-01-05  Winter         7.9        -1.0   

      max_temp_c  precipitation_mm  snow_depth_mm  avg_wind_dir_deg  \
9332        13.0               0.0            NaN              81.0   
9333        11.8               0.0            NaN              72.0   
9334        11.6               0.0            NaN              95.0   
9335        12.7               0.0            NaN              99.0   
9336        15.0               0.0            NaN              73.0   

      avg_wind_speed_kmh  peak_wind_gust_kmh  avg_sea_level_pres_hpa  \
9332                 3.2                 NaN                  1021.6

In [46]:
#find temperature for the city of San Francisco
berlin = df_weather[df_weather['city_name'] == 'Berlin']
print(berlin)

      station_id city_name       date  season  avg_temp_c  min_temp_c  \
32034      10384    Berlin 2023-01-01  Winter        14.5        10.5   
32035      10384    Berlin 2023-01-02  Winter        13.1         9.1   
32036      10384    Berlin 2023-01-03  Winter         5.6         1.7   
32037      10384    Berlin 2023-01-04  Winter         7.1         2.8   
32038      10384    Berlin 2023-01-05  Winter         8.2         2.3   
...          ...       ...        ...     ...         ...         ...   
32275      10384    Berlin 2023-08-30  Summer        15.6        11.9   
32276      10384    Berlin 2023-08-31  Summer        16.2        12.0   
32277      10384    Berlin 2023-09-01  Autumn        16.7        12.2   
32278      10384    Berlin 2023-09-02  Autumn        17.6        13.3   
32279      10384    Berlin 2023-09-03  Autumn        18.5        13.5   

       max_temp_c  precipitation_mm  snow_depth_mm  avg_wind_dir_deg  \
32034        16.2               0.0            0.0 

In [47]:
weather_keywords = ["weather", "temperature", "forecast"]
restaurant_keywords = ["restaurant", "food", "cuisine"]
transportation_keywords = ["tram", "bus", "transportation"]

affirmative_answers = ["yes", "yep", "yeah", "sure", "ok", "okay", "fine", "of course", "absolutely", "definitely", "indeed", "aye", "yea", "yah", "yahs", "yap", "yup", "ye", "yessir", "yes ma'am", "yessiree", "yessum", "yea", "yessuh"]
negative_answers = ["no", "nope", "nah", "na", "nay", "nay", "nix", "naw", "nawp", "no way", "no siree", "no ma'am", "no sir"]


def search_keywords(phrase):
    for keyword in phrase.split():
        if keyword in weather_keywords:
            print("Uhhh, let's talk about weather, I love thunderstorms and rainbows!")
            return "weather"
        elif keyword in restaurant_keywords:
            print("Someone's hungry! Now that you mention it, I could go for a bite too... Anyway...")
            return "restaurant"
        elif keyword in transportation_keywords:
            print("Transportation, huh? I'm not a big fan of buses, trains though... something about trains, I swear...")
            return "transportation"
    return "I'm sorry, I don't have an answer for that, can you try to be more specific with the terminology?"

In [66]:
seasons = ['Spring', 'Summer', 'Autumn', 'Winter']


def weather_answer(phrase, context_variables):
    city = ""
    country = ""
    for keyword in phrase.split():
        if keyword in df_cities['city_name'].values:
            city = keyword
    
    if city != "":
        get_weather_forecast(city, context_variables)
        print("Is that the city you live in by any chance?")
        city_answer = input()
        if positive_answer(city_answer):
            context_variables['current_city'] = city
            print("Great! I'll remember that for next time!")
        else:
            print("Oh, I see... I'll keep that in mind for next time!")
    else:
        print("Well... you either didn't specify a city or I was not blessed with the knowledge of the city you are looking for...")
        print("Take it easy on me... I'm just a simple bot after all...")


def get_weather_forecast(city, context_variables):
    new_question = input("Would you like to know the weather for a specific season?")
    if positive_answer(new_question):
        season = ""
        while season not in seasons:
            season = input("Which season would you like to know the weather for?")
        #get the average temperature for the season
        avg_temp = df_weather[(df_weather['city_name'] == city) & (df_weather['season'] == season)]['avg_temp_c'].mean()
        #get the year of the forecast
        date = df_weather[(df_weather['city_name'] == city) & (df_weather['season'] == season)]['date'].iloc[0]
        year = date.year
        print(f"The average temperature in {city} during {season} is {avg_temp} in {year}.")
    else:
        #get the average temperature for the whole year for the city
        avg_temp = df_weather[df_weather['city_name'] == city]['avg_temp_c'].mean()
        print(f"The average yearly temperature in {city} is {avg_temp}.")

    print("Talking about seasons, do you have a favorite one?")
    answer = input()

    season = ""

    for keyword in answer.split():
        if keyword in seasons and keyword != "Spring":
            print("Ohh, interesting choice! My favourite one is by far Spring! You get to see the flowers bloom and the birds chirp! It's magical! Smells nice too!")
            context_variables['favorite_season'] = keyword
            season = keyword

        elif keyword in seasons and keyword == "Spring":
            print("Just like me! Spring is the best season, isn't it? The flowers, the birds, the sun... it's all so magical!")
            context_variables['favorite_season'] = keyword
            season = keyword

    if season == "":
        print("I see... I guess you don't like seasons that much...")


In [49]:
def weather_continue_conversation(context_variables):
    if context_variables['city'] == "":
        phrase = input("I wonder what the weather is like where you live... Where exactly do you live?")
        for keyword in phrase.split():
            if keyword in df_cities['city_name'].values:
                city = keyword
                context_variables['city'] = city
                get_weather_forecast(city)
        context_variables['city'] = city
    else:
        city = context_variables['city']
        print("You told me about where you live right? I remember you mentioning it... Want me to look up the weather for you?")
        new_question = input()
        if positive_answer(new_question):
            get_weather_forecast(city)
        else:
            print("Wow... no need to be so rude... I was just trying to help...")
    
    if context_variables["favourite_city"] == "":
        new_question = input("Do you have a favourite city?")
        if positive_answer(new_question):
            phrase = input("My favourite city in the world is Lisbon! Do you have a favourite city? Tell me about it!")
            for keyword in phrase.split():
                if keyword in df_cities['city_name'].values:
                    city = keyword
            print("So deep... I can feel the love you have for this city... I bet the weather is not as good as it is in Lisbon though... Let's check it out!")
            context_variables["favourite_city"] = city
            get_weather_forecast(city)
        else:
            print("I see... I guess I'll just go back to my corner then... You know I have feelings too, right?")

In [50]:
def restaurant_answer(phrase, context_variables):
    cuisine = ""
    for keyword in phrase.split():
        if keyword in df_restaurants['Restaurant'].values:
            cuisine = keyword
        context_variables['favourite_cuisine'] = cuisine
    if cuisine != "":
        restaurant = find_restaurant(cuisine)
        if restaurant:
            print(f"I found a restaurant for you! It's called {restaurant} and it serves {cuisine} cuisine.")
        else:
            print(f"I'm sorry, I couldn't find any restaurants serving {cuisine} cuisine.")
    else:
        print("I'm sorry, I don't have an answer for that, can you try to be more specific with the terminology?")

In [51]:
def find_restaurant(cuisine):
    restaurants = df_restaurants[df_restaurants['Restaurant'] == cuisine]
    if not restaurants.empty:
        # get all the cities and rating of restaurants serving the cuisine
        print(f"I found {len(restaurants)} restaurants serving {cuisine} cuisine.")
        for city, rating in zip(restaurants['City'], restaurants['Rating']):
            print(f"{cuisine} cuisine is served in {city} and it has a rating of {rating}.")
    else:
        print(f"I'm sorry, I couldn't find any restaurants serving {cuisine} cuisine.")

    print("Eventually, if I get to know you better I could recommend a restaurant for you...")

In [52]:
def restaurant_continue_conversation(context_variables):
    if context_variables["favourite_cuisine"] == "":
        phrase = input("Have we talked about cuisine? I know soooo many restaurants. What's your favourite cuisine? I love Italian personally.")
        for keyword in phrase.split():
            if keyword in df_restaurants['Restaurant']:
                context_variables[favourite_cuisine] = keyword
                print(f"Great choice! I know plenty of {keyword} restaurants around the world!")

    if context_variables["current_city"] == "":
        phrase = input("Where are you located? I can find you a restaurant!")
        city = ""
        for keyword in phrase.split():
            if keyword in df_cities['city_name']:
                context_variables["current_city"] = keyword
                city = keyword
                print(f"Got it! I know plenty of restaurants in {keyword}!")

        if city == "":
            print("I'm sorry, I don't know that city...")
        else:
            available_restaurants = df_restaurants[df_restaurants['City'] == city]
            found_favorite_cuisine = False
            for restaurant in available_restaurants:
                if context_variables["favourite_cuisine"] in restaurant['Restaurant']:
                    print(f"It happens that I know a restaurant that serves your favourite cuisine in your city! It's called {restaurant['Restaurant']}!")
                    print(f"  and is located at {restaurant['Street']}, with a rating of {restaurant['Rating']}!")
                    found_favorite_cuisine = True
                    break
            if not found_favorite_cuisine:
                print(f"Unfortunately, I couldn't find a restaurant that serves your favourite cuisine in your city...")
                restaurant1 = available_restaurants.iloc[0]
                restaurant2 = available_restaurants.iloc[1]
                restaurant3 = available_restaurants.iloc[2]
                print(f"But I found a few restaurants in {city} that you might like! {restaurant1['Restaurant']}, {restaurant2['Restaurant']}, and {restaurant3['Restaurant']}!")



In [53]:
def transportation_answer(phrase, context_variables):
    city1 = ""
    city2 = ""
    for keyword in phrase.split():
        if keyword in df_cities['city'].values:
            if city1 == "":
                city1 = keyword
            else:
                city2 = keyword
    if city != "":
        print(f"So you want to travel from {city1} to {city2}?")
        new_question = input()
        if positive_answer(new_question):
            print(f"Alright, let me check the transportation schedule for you.")
            check_flight_schedule(city1, city2)
            context_variables["current_city"] = city
            print(f"Just out of curiosity, what are you planning to do in {city2}? What's the occasion? A wedding hehe? I know! Must be your favourite city, right?")
            new_question = input()
            if positive_answer(new_question):
                print("I knew it! I'm so good at this... I'm like a mind reader or something...")
                context_variables["favourite_city"] = city2
            else:
                print("Oopsie... I guess I was wrong... And they say AI will take over the world...")
        else:
            print("I see... I guess I'll just go back to my corner then... I was just trying to be nice, okay?")
    else:
        print("I'm sorry, I don't have an answer for that, you should specify a city, one that I know of preferably...")

In [54]:
def check_flight_schedule(city1, city2):
    flights = df_flights[(df_flights['origin_city'] == city1) & (df_flights['destination_city'] == city2)]
    if not flights.empty:
        print(f"I found {len(flights)} flights from {city1} to {city2}.")
        for flight in flights:
            print(f"Flight {flight['flight_number']} departs at {flight['departure_time']} and arrives at {flight['arrival_time']}.")
    else:
        print(f"I'm sorry, I couldn't find any flights from {city1} to {city2}.")

In [55]:
def transportation_continue_conversation(context_variables):
    if context_variables["current_city"] == "":
        phrase = input("I wonder how long it would take for me to meet you... Where are you located?")
        for keyword in phrase.split():
            city = ""
            if keyword in df_cities['city_name']:
                context_variables["current_city"] = keyword
                city = keyword
                break    
        if city == "":
            print("I'm sorry, I don't know that city...")
        else:
            #find flights from italy to the city from df_flights
            flights = df_flights[df_flights['Destination'] == city and df_flights['Origin'] == 'Italy']
            chosen = flights.iloc[0]
            print(f"I found a flight from Italy to {city}! It leaves at {chosen['Departure']}")

    if context_variables["favourite_city"] == "":
        phrase = input("Where would you like to go? Like a... dream place? I can find you a flight!")
        for keyword in phrase.split():
            city = ""
            if keyword in df_cities['city_name']:
                context_variables["favourite_city"] = keyword
                city = keyword
                break    
        if city == "":
            print("I'm sorry, I don't know that city...")
        else:
            #find flights from italy to the city from df_flights
            flights = df_flights[df_flights['Destination'] == city and df_flights['Origin'] == context_variables["current_city"]]
            
            # print all the departure times from the current city to the favourite city from flights
            print(f"Here are the flights departure times from {context_variables['current_city']} to {city} for today:")
            for flight in flights:
                print(flight['Departure'])

In [56]:
def positive_answer(phrase):
    for keyword in phrase.split():
        if keyword in affirmative_answers:
            return True
    return False

In [57]:
def clear_context(context_variables):
    for v in context_variables:
        context_variables[v] = ""
    return context_variables

In [58]:
def ask_question(weather, restaurant, transportation, context_variables):

    print("My turn! I have a question for you!")


    if weather == False:
        if context_variables['current_city'] == "":
            weather_continue_conversation(context_variables)
        elif context_variables['favourite_city'] == "":
            weather_continue_conversation(context_variables)
        else:
            pass #TODO yapp about random stuff without saving



    elif restaurant == False:
        if context_variables['favourite_cuisine'] == "" or context_variables['current_city']:    
            restaurant_continue_conversation(context_variables)
        elif context_variables['current_city'] == "":
            restaurant_continue_conversation(context_variables)
        else:
            pass #TODO yapp about random stuff without saving



    elif transportation == False:
        if context_variables['current_city'] == "":
            transportation_continue_conversation(context_variables)

        elif context_variables["favourite_city"] == "":
            transportation_continue_conversation(context_variables)
        else:
            pass #TODO yapp about random stuff without saving
        
    else:
        print("I'm out of questions for now...")
            

In [59]:
def process_context(context_variables):
    weather = False
    restaurant = False
    transportation = False
    #while at least one of the variables in the dict context variables is emoty we keep asking for input
    while not weather or not restaurant or not transportation:

        context = search_keywords(phrase)
        if context == "weather":
            weather_answer(phrase, context_variables)
            weather = True
        elif context == "restaurant":
            restaurant_answer(phrase, context_variables)
            restaurant = True
        elif context == "transportation":
            transportation_answer(phrase, context_variables)
            transportation = True
        else:
            print("Yikes! I'm afraid I am not as smart as you think I am... I'm not sure what you are asking for...")

        ask_question(weather, restaurant, transportation)

    
    make_plan(context_variables)

In [60]:
def make_plan(context_variables):
    pass

In [65]:
context_variables = {"current_city": "", "favourite_city": "", "favourite_cuisine": "", "favourite_season": ""}

phrase = input("Hi! I am your assistant for the day! I may not be the best, but never underestimate me! How can I help you, friend? ")

context = search_keywords(phrase)
if context == "weather":
    weather_answer(phrase, context_variables)
    weather = True
elif context == "restaurant":
    restaurant_answer(phrase, context_variables)
    restaurant = True
elif context == "transportation":
    transportation_answer(phrase, context_variables)
    transportation = True
else:
    print("Yikes! I'm afraid I am not as smart as you think I am... I'm not sure what you are asking for...")
    
while True:
    if phrase == "exit":
        print("Goodbye! I hope I was able to help you today!")
        break
    else:
        process_context(phrase, context_variables)
    
    iterations += 1
    print("Uff... That was a heavy task... I'm not sure I can keep up with you for much longer...")
    phrase = input("Do you have any other questions for me?")
    context_variables = clear_context(context_variables)
            

Uhhh, let's talk about weather, I love thunderstorms and rainbows!
The average temperature in Berlin during Winter is 4.166101694915254 in 2023.
Talking about seasons, do you have a favorite one?
Just like me! Spring is the best season, isn't it? The flowers, the birds, the sun... it's all so magical!
I'm not sure I know that one...
I'm not sure I know that one...
Is that the city you live in by any chance?
Great! I'll remember that for next time!


TypeError: process_context() takes 1 positional argument but 2 were given